In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

df = pd.read_csv('../input/forest-cover-type-kernels-only/train.csv.zip')

df.head()

In [ ]:
df.insert(0, "nn", np.sqrt(df['Horizontal_Distance_To_Hydrology']**2 +df['Vertical_Distance_To_Hydrology']**2))
df.head()

In [ ]:
df.shape

In [ ]:
####### modified: Id column must be dropped before assigning X and y
df = df.drop(columns="Id")

In [ ]:
#input-features encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

X = df.values[:, 0:-1]
y = df.values[:, -1] - 1

In [ ]:
#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=5),
                        LogisticRegression(random_state=1))

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

In [ ]:
#Standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=5)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

from sklearn.model_selection import validation_curve

forest = RandomForestClassifier(n_estimators=30, random_state=1)

param_range = [2, 5, 8, 10, 15, 25, 30]

train_scores, test_scores = validation_curve(
                estimator=forest, 
                X=X_train, 
                y=y_train, 
                param_name='max_depth', #here you write the parameter name as the docomentation of it says -> knn use n_neighbors
                param_range=param_range,
                cv=10)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

forest = RandomForestClassifier(random_state=1)


max_depth_range = [7, 10, 15, 32, 40, 47]

n_estimators_range = [2, 10, 15,  30,  50, 60]

param_grid = [ {'max_depth': max_depth_range, 'n_estimators': n_estimators_range} ]

gs = GridSearchCV(estimator=forest, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5)

gs = gs.fit(X_train_std, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
clf = gs.best_estimator_


print('Test accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
# Preprocess Test Data
df_submit = pd.read_csv('../input/forest-cover-type-kernels-only/test.csv.zip')
id_submit = df_submit.values[:, 0]
df_submit = df_submit.drop(columns='Id')

## put your feature engineering here but for df_submit ##
#-------------------------------------------------------#
#########################################################
df_submit.insert(0, "nn", np.sqrt(df_submit['Horizontal_Distance_To_Hydrology']**2 +df_submit['Vertical_Distance_To_Hydrology']**2))


X_submit = df_submit.values
X_submit_std = sc.transform(X_submit)


print("Evaluation..")
y_submit = clf.predict(X_submit_std)
###### modified: it was written gs but your chosen classifier is clf

submission = {'Id':id_submit, 'Cover_Type': (y_submit + 1).astype(int)}
df_submission = pd.DataFrame(submission)
df_submission.to_csv('submission.csv', index=False)
